<a href="https://colab.research.google.com/github/malongfei1993/longTextSummary_chatglm3-6b-32k/blob/main/chatglm3_peft_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install peft

  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.17.3 which is incompatible.


In [3]:
!pip install sentencepiece
!pip install bitsandbytes
!pip install accelerate
!pip install peft
!pip install datasets

  Using cached huggingface_hub-0.20.3-py3-none-any.whl (330 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.20.3 which is incompatible.


In [4]:
# !pip uninstall transformers

这里需要指定下transformers版本

In [5]:
# !pip install transformers==4.34.0

In [6]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [7]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

In [10]:
# def compare_chatglm_results(query):
#     base_response, base_history = base_model.chat(tokenizer, query,temperature=0.1,history=[])

#     # inputs = tokenizer(query, return_tensors="pt").to(0)
#     # ft_out = model.generate(**inputs, max_new_tokens=512,temperature=0.1)
#     # ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
#     print(base_response)
#     # print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
#     return base_response

In [11]:
test_text = '''这是数据：{
"ID": NaN,
"General characteristics": "Unit scope",
"Information": NaN,
"Value": "Cover lens+Panel + backlight + T-CON + Optical bonding + Touch function",
"Response": NaN
},
{
"ID": "1.1",
"General characteristics": "Display size",
"Information": "Disassociated Center Stack Display w/ Oversized Lens design",
"Value": "12.9" (Supplier to suggest better size)",
"Response": "2023/10/25: 12.8" FHD"
},
{
"ID": 1.2,
"General characteristics": "Orientation",
"Information": "Portrait / Landscape\u2026",
"Value": "Landscape",
"Response": NaN
},----------
请回复如下格式：
[{
"参数":"",
"需求规格":"",
""供应商反馈""：""
}]
你的任务：
提取对角线尺寸的参数规格需求信息和供应商反馈信息。
下面是你的回复(使用英文)：'''

In [28]:
response, history = base_model.chat(tokenizer,test_text,temperature=0.1,history=[])
response

'{\n"参数": "Display size",\n"需求规格": "12.9" (Supplier to suggest better size)",\n"供应商反馈": "2023/10/25: 12.8" FHD"\n}'

#**问题出现在下面这段加载peft这里，只要执行完这段代码，原来模型的输出就变成与微调后的输出一样**

In [24]:

from peft import PeftModel, PeftConfig

epochs = 5
peft_model_path = f"drive/MyDrive/fineTuningModel/models/THUDM/chatglm3-6b-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)
# model = AutoModel.from_pretrained(peft_model_path,device_map="auto",trust_remote_code=True)

In [25]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (embedding): Embedding(
          (word_embeddings): Embedding(65024, 4096)
        )
        (rotary_pos_emb): RotaryEmbedding()
        (encoder): GLMTransformer(
          (layers): ModuleList(
            (0-27): 28 x GLMBlock(
              (input_layernorm): RMSNorm()
              (self_attention): SelfAttention(
                (query_key_value): lora.Linear4bit(
                  (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=4608, bias=False)

In [26]:
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=4096, out_features=4608, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=4608, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
           

**这是微调后的输出**，可以执行下base_model的那行，输出变得与微调后的一样了

In [27]:
response, history = model.chat(tokenizer,test_text,temperature=0.1,history=[])
response

'{\n"参数": "Display size",\n"需求规格": "12.9" (Supplier to suggest better size)",\n"供应商反馈": "2023/10/25: 12.8" FHD"\n}'

In [ ]:
# base_response = compare_chatglm_results(query='''
# 这是数据：{
# "ID": NaN,
# "General characteristics": "Unit scope",
# "Information": NaN,
# "Value": "Cover lens+Panel + backlight + T-CON + Optical bonding + Touch function",
# "Response": NaN
# },
# {
# "ID": "1.1",
# "General characteristics": "Display size",
# "Information": "Disassociated Center Stack Display w/ Oversized Lens design",
# "Value": "12.9" (Supplier to suggest better size)",
# "Response": "2023/10/25: 12.8" FHD"
# },
# {
# "ID": 1.2,
# "General characteristics": "Orientation",
# "Information": "Portrait / Landscape\u2026",
# "Value": "Landscape",
# "Response": NaN
# },----------
# 请回复如下格式：
# [{
# "参数":"",
# "需求规格":"",
# ""供应商反馈""：""
# }]
# 你的任务：
# 提取对角线尺寸的参数规格需求信息和供应商反馈信息。
# 下面是你的回复(使用英文)：


# ''')